# Polarizability using automatic differentiation

Simple example for computing properties using (forward-mode)
automatic differentiation.
For a more classical approach and more details about computing polarizabilities,
see Polarizability by linear response.

In [1]:
using DFTK
using LinearAlgebra
using ForwardDiff

# Construct PlaneWaveBasis given a particular electric field strength
# Again we take the example of a Helium atom.
function make_basis(ε::T; a=10., Ecut=30) where {T}
    lattice=T(a) * I(3)  # lattice is a cube of $a$ Bohrs
    # Helium at the center of the box
    atoms     = [ElementPsp(:He, psp=load_psp("hgh/lda/He-q2"))]
    positions = [[1/2, 1/2, 1/2]]

    model = model_DFT(lattice, atoms, positions, [:lda_x, :lda_c_vwn];
                      extra_terms=[ExternalFromReal(r -> -ε * (r[1] - a/2))],
                      symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid=[1, 1, 1])  # No k-point sampling on isolated system
end

# dipole moment of a given density (assuming the current geometry)
function dipole(basis, ρ)
    @assert isdiag(basis.model.lattice)
    a  = basis.model.lattice[1, 1]
    rr = [a * (r[1] - 1/2) for r in r_vectors(basis)]
    sum(rr .* ρ) * basis.dvol
end

# Function to compute the dipole for a given field strength
function compute_dipole(ε; tol=1e-8, kwargs...)
    scfres = self_consistent_field(make_basis(ε; kwargs...); tol)
    dipole(scfres.basis, scfres.ρ)
end;

With this in place we can compute the polarizability from finite differences
(just like in the previous example):

In [2]:
polarizability_fd = let
    ε = 0.01
    (compute_dipole(ε) - compute_dipole(0.0)) / ε
end

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770798808239                   -0.53    8.0         
  2   -2.772136829609       -2.87       -1.31    1.0    151ms
  3   -2.772169914245       -4.48       -2.56    1.0    134ms
  4   -2.772170711950       -6.10       -3.58    2.0    154ms
  5   -2.772170722180       -7.99       -4.05    1.0    160ms
  6   -2.772170722954       -9.11       -5.13    1.0    137ms
  7   -2.772170723010      -10.25       -5.16    2.0    178ms
  8   -2.772170723015      -11.33       -6.11    1.0    144ms
  9   -2.772170723015      -12.92       -6.68    2.0    184ms
 10   -2.772170723015      -13.99       -7.31    1.0    151ms
 11   -2.772170723015      -14.88       -8.16    2.0    180ms
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770653161211                   -0.53    8.0         
  2   -2.7

1.7735580405222033

We do the same thing using automatic differentiation. Under the hood this uses
custom rules to implicitly differentiate through the self-consistent
field fixed-point problem.

In [3]:
polarizability = ForwardDiff.derivative(compute_dipole, 0.0)
println()
println("Polarizability via ForwardDiff:       $polarizability")
println("Polarizability via finite difference: $polarizability_fd")

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -2.770827783044                   -0.52    9.0         
  2   -2.772062650687       -2.91       -1.32    1.0    130ms
  3   -2.772083101393       -4.69       -2.47    1.0    159ms
  4   -2.772083349760       -6.60       -3.17    1.0    134ms
  5   -2.772083417186       -7.17       -4.08    2.0    152ms
  6   -2.772083417692       -9.30       -4.58    1.0    175ms
  7   -2.772083417810       -9.93       -5.88    2.0    157ms
  8   -2.772083417811      -12.82       -6.51    2.0    161ms
  9   -2.772083417811      -13.85       -6.84    1.0    176ms
 10   -2.772083417811   +  -14.18       -7.82    2.0    168ms
 11   -2.772083417811   +  -14.57       -8.94    1.0    153ms

Polarizability via ForwardDiff:       1.7725349684302627
Polarizability via finite difference: 1.7735580405222033
